In [2]:
# Librerias, archivos y configuración

import warnings
import os
import polars as pl
import pyarrow
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [14]:
# Se importan los datos

df = pl.read_csv('../../../../../../Documents/Databases//trafic_mad/20.01.csv', sep=';', infer_schema_length=0)

In [15]:
df.head()

id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
str,str,str,str,str,str,str,str,str
"""1001""","""2020-01-01 00:...","""M30""","""264""","""0""","""0""","""57""","""N""","""5"""
"""1001""","""2020-01-01 00:...","""M30""","""252""","""0""","""0""","""68""","""N""","""5"""
"""1001""","""2020-01-01 00:...","""M30""","""252""","""0""","""0""","""65""","""N""","""5"""
"""1001""","""2020-01-01 00:...","""M30""","""744""","""1""","""0""","""59""","""N""","""5"""
"""1001""","""2020-01-01 01:...","""M30""","""1212""","""2""","""0""","""62""","""N""","""5"""


In [21]:
df.shape

(11577408, 9)

In [22]:
# Realmente solo me interesan las columnas id, fecha, hora, longitud y latitud. Se borra el resto

4